In [2]:
import sys, os, csv, re
import collections
import subprocess

from lxml import etree
from pprint import pprint

%load_ext autoreload
%autoreload 2
import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.lib import Transcription, monad_set

#from etcbc.mql import MQL
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.5.21
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [31]:
API=fabric.load('etcbc4b', '--', 'mql', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        otype nu ps gn vs vt prs ls lex g_cons g_word_utf8 g_cons_utf8
        function txt
        book chapter verse label sp kind typ 
    ''',
    ''' functional_parent
    '''),
    "prepare": prepare,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))
#Q = MQL(API)

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.00s USING main  DATA COMPILED AT: 2015-12-17T15-56-05
  0.00s DETAIL: COMPILING a: UP TO DATE
  0.01s DETAIL: keep main: G.node_anchor_min
  0.01s DETAIL: keep main: G.node_anchor_max
  0.01s DETAIL: keep main: G.node_sort
  0.01s DETAIL: keep main: G.node_sort_inv
  0.01s DETAIL: keep main: G.edges_from
  0.01s DETAIL: keep main: G.edges_to
  0.01s DETAIL: keep main: F.etcbc4_db_otype [node] 
  0.01s DETAIL: keep main: F.etcbc4_ft_g_word_utf8 [node] 
  0.01s DETAIL: keep main: F.etcbc4_sft_book [node] 
  0.01s DETAIL: keep main: F.etcbc4_sft_chapter [node] 
  0.01s DETAIL: keep main: F.etcbc4_sft_verse [node] 
  0.01s DETAIL: clear main: F.etcbc4_ft_lex_utf8 [node] 
  0.01s DETAIL: clear main: F.etcbc4_ft_trailer_utf8 [node] 
  0.01s DETAIL: clear main: F.etcbc4_kq_g_qere_utf8 [node] 
  0.01s DETAIL: clear main: F.etcbc4_kq_qtrailer_utf8 [node] 
  0.01s DETAIL: clear main: F.etcbc4_ph_phono [node] 
  0.0

All bet preposition constructions with infinitive hifil or nifal in the HB, at the beginning of a clause.
F. we need verbal stem (vs) 
F. verbal tense (vt)
value for vt = infc
value for vs = nifal, hifil

otype = word
InfC, value 

In [60]:
infc_lst = []


def find_infc():
    '''Finds the infinitive construct phrase with verbal stem hifil or nifal with prepostion b.'''
    for node in NN():
        if F.otype.v(node) == 'word': 
            if F.vt.v(node) == 'infc' and F.vs.v(node) == 'hif' or F.vs.v(node) == 'nif':
                phrases = L.u('phrase', node)
                b_found = False
                for word in L.d('word', phrases):
                    if word == node and b_found:   
                        infc_lst.append(word)
                        break
                    b_found = F.lex.v(word) == 'B'

            
find_infc()

In [63]:
words = [F.g_cons_utf8.v(n) for n in infc_lst]
len(words)

117

In [87]:
def is_start(n):
    '''Finds the second word in the clause. The infc is in the second position of the clause'''
    c = L.u('clause', n)
    words = list(L.d('word', c))
    return len(words) > 1 and words[1] == n

In [110]:
data_lst = []
for n in infc_lst: 
    book = L.u('book', n)
    verse = L.u('verse', n)
    clause = L.u('clause', n)
    data_lst.append([
            str(n), 
            str(is_start(n)), 
            F.g_cons.v(n), 
            T.book_name(book), 
            str(F.chapter.v(verse)), 
            str(F.verse.v(verse)),
            T.words(L.d('word', clause)).strip('\n')
    ])

In [111]:
with open("b-Inf.csv", 'w') as f:
    header = ['node', 'start', 'verb', 'book', 'chapter', 'verse', 'clause']
    f.write('{}\n'.format(','.join(header)))

    for item in data_lst:
        f.write('{}\n'.format(','.join(item)))

In [93]:
for (node, start, verb, book, chapter, verse, clause) in data_lst:
    print('{} {} {}'.format(verb, start, clause))

הבראם True בְּהִבָּֽרְאָ֑ם 
המלו True בְּהִמֹּלֹ֖ו בְּשַׂ֥ר עָרְלָתֹֽו׃

המלו True בְּהִ֨מֹּלֹ֔ו אֵ֖ת בְּשַׂ֥ר עָרְלָתֹֽו׃

הולד True בְּהִוָּ֣לֶד לֹ֔ו אֵ֖ת יִצְחָ֥ק בְּנֹֽו׃

העטיף True בְהַעֲטִ֥יף הַצֹּ֖אן 
האבקו True בְּהֵֽאָבְקֹ֖ו עִמֹּֽו׃

המול True בְּהִמֹּ֥ול לָ֨נוּ֙ כָּל־זָכָ֔ר 
הקשׁתה True בְהַקְשֹׁתָ֖הּ 
הוציאך True בְּהֹוצִֽיאֲךָ֤ אֶת־הָעָם֙ מִמִּצְרַ֔יִם 
הכתי True בְּהַכֹּתִ֖י בְּאֶ֥רֶץ מִצְרָֽיִם׃

הכבדי True בְּהִכָּבְדִ֣י בְּפַרְעֹ֔ה בְּרִכְבֹּ֖ו וּבְפָרָשָֽׁיו׃

הוציאי True בְּהֹוצִיאִ֥י אֶתְכֶ֖ם מֵאֶ֥רֶץ מִצְרָֽיִם׃

היטיבו True בְּהֵיטִיבֹ֛ו אֶת־הַנֵּרֹ֖ת 
העלת True בְהַעֲלֹ֨ת אַהֲרֹ֧ן אֶת־הַנֵּרֹ֛ת בֵּ֥ין הָעֲרְבַּ֖יִם 
העלות True בְהֵעָלֹ֤ות הֶֽעָנָן֙ מֵעַ֣ל הַמִּשְׁכָּ֔ן 
הוציאי True בְּהֹוצִיאִ֥י אֹותָ֖ם מֵאֶ֣רֶץ מִצְרָ֑יִם 
הקרבם True בְּֽהַקְרִבָם֩ אֵ֨שׁ זָרָ֜ה לִפְנֵ֤י יְהוָה֙ בְּמִדְבַּ֣ר סִינַ֔י 
העלתך True בְּהַעֲלֹֽתְךָ֙ אֶת־הַנֵּרֹ֔ת 
האריך False וּבְהַאֲרִ֧יךְ הֶֽעָנָ֛ן עַל־הַמִּשְׁכָּ֖ן יָמִ֣ים רַבִּ֑ים 
האריך True בְּהַאֲרִ֨יךְ הֶעָנָ֤ן עַל־הַמִּשְׁכָּן֙ 
העלתו